## D2 missing some routes [PR](https://github.com/cal-itp/data-analyses/issues/1582)

In [1]:
import _ct_district_data_prep
import _transit_routes_on_shn
# import _ct_district_visuals
import _report_utils
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

In [2]:
import google.auth

credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [3]:
district = "02 - Redding"

In [4]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [5]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [6]:
portfolio_organization_names = list(operator_df.portfolio_organization_name.unique())

In [7]:
portfolio_organization_names

['Modoc Transportation Agency',
 'Siskiyou County',
 'Redding Area Bus Authority',
 'Tehama County',
 'Trinity County',
 'Plumas Transit Systems',
 'Lassen Transit Service Agency']

In [8]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

In [9]:
operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    storage_options={"token": credentials.token},
)[["portfolio_organization_name", "service_date", "recent_combined_name", "geometry"]]

operator_route_gdf = operator_route_gdf.loc[
    operator_route_gdf.portfolio_organization_name.isin(portfolio_organization_names)
]

In [10]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

In [11]:
# Save everything out for now
intersection_gdf = gpd.read_parquet(
        f"{GCS_FILE_PATH}transit_route_intersect_shn_50_gtfs_digest.parquet",
        storage_options={"token": credentials.token},
    )
    

In [12]:
open_data_portal_df = pd.read_parquet(
        f"{GCS_FILE_PATH}transit_route_shn_open_data_portal_50.parquet",
        storage_options={"token": credentials.token},
    )

In [13]:
open_data_portal_df.head(1)

,portfolio_organization_name,recent_combined_name,shn_route,District,pct_route_on_hwy_across_districts,on_shs
0,Alameda-Contra Costa Transit District,10 E. 14th St. - Mission,"185, 112, 238",4,39.0,Y


In [14]:
open_data_portal_df.loc[open_data_portal_df.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

,portfolio_organization_name,recent_combined_name,shn_route,District,pct_route_on_hwy_across_districts,on_shs
1742,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"32, 273, 5, 99, 36, 44","2, 3",97.8,Y


In [15]:
open_data_df2 = open_data_portal_df.loc[
        (open_data_portal_df.pct_route_on_hwy_across_districts > 0)
        & (open_data_portal_df.District.str.contains(str(district_int)))
    ]

In [16]:
open_data_df3 = open_data_df2[open_data_df2['District'].apply(lambda x: str(district_int) in x.split(', '))]

In [17]:
open_data_df3.District.unique()

array(['3, 5, 2, 4, 7', '2, 3', '2', '2, 1'], dtype=object)

In [18]:
open_data_df3.loc[open_data_df3.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

,portfolio_organization_name,recent_combined_name,shn_route,District,pct_route_on_hwy_across_districts,on_shs
1742,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"32, 273, 5, 99, 36, 44","2, 3",97.8,Y


In [19]:
open_data_df2.District.unique()

array(['3, 5, 2, 4, 7', '12', '12, 7', '6, 12, 11, 7', '11, 12, 7',
       '12, 6, 11, 4, 7, 10', '2, 3', '12, 3, 6, 11, 7, 10', '12, 8, 7',
       '2', '12, 8', '2, 1'], dtype=object)

In [20]:
open_data_df2.loc[open_data_df2.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

,portfolio_organization_name,recent_combined_name,shn_route,District,pct_route_on_hwy_across_districts,on_shs
1742,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"32, 273, 5, 99, 36, 44","2, 3",97.8,Y


In [21]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(0, str(district_int))
)
# district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [24]:
transit_route_shs_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column                                                   Non-Null Count  Dtype   
---  ------                                                   --------------  -----   
 0   Portfolio Organization Name                              41 non-null     object  
 1   Route                                                    41 non-null     object  
 2   geometry                                                 41 non-null     geometry
 3   State Highway Network Route                              41 non-null     object  
 4   District                                                 41 non-null     object  
 5   Percentage of Transit Route on SHN Across All Districts  41 non-null     float64 
dtypes: float64(1), geometry(1), object(4)
memory usage: 2.2+ KB


In [22]:
transit_route_shs_table.loc[
    transit_route_shs_table["Portfolio Organization Name"]
    == "Redding Area Bus Authority"
].sort_values(by=["Route"])

,Portfolio Organization Name,Route,State Highway Network Routes in District 2,Percentage of Transit Route on SHN Across All Districts
11,Redding Area Bus Authority,Churn Creek/Knightson/Airport,"44, 5, 273",44.4
12,Redding Area Bus Authority,Route 1,"299, 5, 151, 273",41.2
13,Redding Area Bus Authority,Route 11,"44, 299, 5, 273",21.5
14,Redding Area Bus Authority,Route 14,"44, 299, 5, 273",16.4
15,Redding Area Bus Authority,Route 299X,"5, 299, 89, 44, 273",97.2
16,Redding Area Bus Authority,Route 3,273,17.6
17,Redding Area Bus Authority,Route 4,"44, 5, 273",5.7
18,Redding Area Bus Authority,Route 7,"299, 5, 273",55.2
19,Redding Area Bus Authority,Route 9,"44, 5, 273",55.6
20,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"99, 5, 44, 36, 273",97.8


In [23]:
operator_route_gdf.loc[
    operator_route_gdf.portfolio_organization_name == "Redding Area Bus Authority"
].drop(columns=["geometry"]).sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).head(
    20
)

,portfolio_organization_name,service_date,recent_combined_name
64919,Redding Area Bus Authority,2025-06-11,Churn Creek/Knightson/Airport
64920,Redding Area Bus Authority,2025-06-11,Route 1
64924,Redding Area Bus Authority,2025-06-11,Route 11
64925,Redding Area Bus Authority,2025-06-11,Route 14
64926,Redding Area Bus Authority,2025-06-11,Route 299X
64921,Redding Area Bus Authority,2025-06-11,Route 3
64922,Redding Area Bus Authority,2025-06-11,Route 4
64923,Redding Area Bus Authority,2025-06-11,Route 7
64930,Redding Area Bus Authority,2025-06-11,Route 9
64928,Redding Area Bus Authority,2025-06-11,Route 99X/Amtrak Thruway Route 3


## Can't find Route 19 and Route 99X in the `_ct_district_data_prep.final_transit_route_shs_outputs` area

In [25]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

subset = [
    "service_date",
    "geometry",
    "portfolio_organization_name",
    "recent_combined_name",
    "route_id",
]
op_geography_df = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    storage_options={"token": credentials.token},
)[subset]

In [26]:
op_geography_df.portfolio_organization_name.unique()

array(['Tulare County Regional Transit Agency',
       'San Luis Obispo Regional Transit Authority',
       'Marin County Transit District', 'City of Baldwin Park',
       'City of Inglewood', 'City of Auburn', 'City of Bell',
       'Angel Island-Tiburon Ferry Company', 'City of Tracy',
       'Santa Barbara Metropolitan Transit District',
       'Los Angeles County Metropolitan Transportation Authority',
       'City of Pasadena', 'Tahoe Transportation District',
       'Sonoma-Marin Area Rail Transit District', 'City of Los Angeles',
       'City of West Hollywood', 'Redwood Coast Transit Authority',
       'Sonoma County Transit Schedule', 'Lake Transit Authority',
       'City of Huntington Park', 'Yuba-Sutter Transit Authority',
       'City of Glendale', 'Yurok Tribe', 'City of Bell Gardens', 'POINT',
       'Modoc Transportation Agency', 'City of Fairfield',
       'City of Arvin', 'Alameda-Contra Costa Transit District',
       'City of Cudahy', 'Emeryville Transportation Mana

In [27]:
raba = op_geography_df.loc[
    op_geography_df.portfolio_organization_name == "Redding Area Bus Authority"
]

### Route 19 is missing but Route 99x is here.

In [28]:
raba[["recent_combined_name", "route_id", "service_date"]].sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).drop_duplicates(subset=["recent_combined_name"])

,recent_combined_name,route_id,service_date
64919,Churn Creek/Knightson/Airport,15,2025-06-11
64920,Route 1,151,2025-06-11
64924,Route 11,159,2025-06-11
64925,Route 14,160,2025-06-11
64926,Route 299X,161,2025-06-11
64921,Route 3,153,2025-06-11
64922,Route 4,154,2025-06-11
64923,Route 7,157,2025-06-11
64930,Route 9,6446,2025-06-11
64928,Route 99X/Amtrak Thruway Route 3,24,2025-06-11


In [29]:
processed_transit_routes = _transit_routes_on_shn.process_transit_routes()

In [30]:
processed_transit_routes.columns

Index(['service_date', 'geometry', 'portfolio_organization_name',
       'recent_combined_name', 'route_length_feet'],
      dtype='object')

In [31]:
raba_only = processed_transit_routes.loc[
    processed_transit_routes.portfolio_organization_name == "Redding Area Bus Authority"
]

In [32]:
raba_only.sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).drop(columns = ["geometry"])

,service_date,portfolio_organization_name,recent_combined_name,route_length_feet
2779,2025-06-11,Redding Area Bus Authority,Churn Creek/Knightson/Airport,74595.123463
2780,2025-06-11,Redding Area Bus Authority,Route 1,59404.153423
2784,2025-06-11,Redding Area Bus Authority,Route 11,58132.163204
2785,2025-06-11,Redding Area Bus Authority,Route 14,60492.082832
2786,2025-06-11,Redding Area Bus Authority,Route 299X,316814.705317
2781,2025-06-11,Redding Area Bus Authority,Route 3,46149.322904
2782,2025-06-11,Redding Area Bus Authority,Route 4,55847.834461
2783,2025-06-11,Redding Area Bus Authority,Route 7,49674.455686
2790,2025-06-11,Redding Area Bus Authority,Route 9,89130.963942
2788,2025-06-11,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,402000.614903


In [33]:
shn_raba = _transit_routes_on_shn.routes_shn_intersection(50,"ct_district_route")

In [34]:
shn_raba  = shn_raba.loc[
    shn_raba.portfolio_organization_name == "Redding Area Bus Authority"
]

In [35]:
shn_raba.columns

Index(['service_date', 'geometry', 'portfolio_organization_name',
       'recent_combined_name', 'route_length_feet', 'shn_route', 'district',
       'highway_feet', 'pct_route_on_hwy'],
      dtype='object')

In [36]:
shn_raba.shape

(45, 9)

In [37]:
route_99 = shn_raba.loc[shn_raba.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

In [38]:
route_99 = route_99.rename(columns={"pct_route_on_hwy": "pct_route_on_hwy_across_districts"})

In [39]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:

    # Aggregate by adding all the districts and SHN to a single row, rather than
    # multiple and sum up the total % of SHN a transit route intersects with
    agg1 = (
        df.groupby(
            [
                "portfolio_organization_name",
                "recent_combined_name",
            ],
            as_index=False,
        )[["shn_route", "district", "pct_route_on_hwy_across_districts"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "district": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy_across_districts": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )

    # Clean up
    agg1.pct_route_on_hwy_across_districts = (
        agg1.pct_route_on_hwy_across_districts.astype(float).round(2)
    )
    return agg1

In [40]:
agg1 = group_route_district(route_99, "sum")

In [41]:
agg1


,portfolio_organization_name,recent_combined_name,shn_route,district,pct_route_on_hwy_across_districts
0,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"99, 5, 44, 36, 32, 273","3.0, 2.0",97.8


In [42]:
agg1.loc[
        agg1.district.astype(str).str.contains(str(district_int))
    ]

,portfolio_organization_name,recent_combined_name,shn_route,district,pct_route_on_hwy_across_districts
0,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"99, 5, 44, 36, 32, 273","3.0, 2.0",97.8


In [43]:
open_data_portal_df = _transit_routes_on_shn.prep_open_data_portal(raba_only)

KeyError: "Columns not found: 'District', 'shn_route', 'pct_route_on_hwy_across_districts'"